In [1]:
import numpy as np
import math as mt


In [4]:
alpha = [104, 128, 135, 139, 150, 153, 162, 168, 195, 198]
beta = [9, 8, 7, 7, 6, 6, 5, 2, 1, 1]
R_Squared = 68644

In [40]:
def calculate_Summation(alpha, beta):
    summ = 0
    for i in range(len(alpha)):
        summ += (alpha[i]*alpha[i])/(beta[i])
        
    return summ

def calculate_Xi_Index(alpha, beta, R_Squared, index, X):
    summation = calculate_Summation(alpha, beta)
    
    for i in range(index):
        summation -= (alpha[i]*alpha[i])/(beta[i])
        R_Squared -= beta[i]*X[i]*X[i]
        
    R = mt.sqrt(R_Squared)
    
    for i in range(index, len(alpha)):
        X[i] = (R*alpha[i])/(beta[i]*mt.sqrt(summation))
    
    return X

def get_Upper_Bound(X):
    return np.matmul(X, alpha)

def get_Lower_Bound(X):
    X = [mt.floor(x) for x in X]
    return np.matmul(X, alpha)
    

X = [0]*len(alpha)

### Generate a greedy variable assignment to try to get a good lower bound on the value of the maximum for this problem. The larger this bound, the better.

To generate a greedy variable assignment, we use the greedy gradient ascent algorithm. We start at X = [0,0,0,0,0,0,0,0,0,0] and increase the value by a factor of the gradient of F(x), that is the alpha list [104, 128, 135, 139, 150, 153, 162, 168, 195, 198]. The step size for each x<sub>i</sub> will depend on the beta[i] factor. Using this method we increase the value of x<sub>i</sub> and stop one iteration before the constraint value goes over 68644.

In [54]:
def calculate_Constraint(X, beta):
    X_Squared = np.square(np.array(X))
    return np.matmul(beta, np.array(X_Squared))

def calculate_Fx(X, alpha):
    return np.dot(alpha, np.array(X))

def calculate_Gradient_Step(step_Size, beta, gradient):
    gradient_Step = []
    for i in range(len(beta)):
        gradient_Step.append((step_Size/beta[i])*gradient[i])
        
    return gradient_Step
        

def greedy_Ascent(alpha, R_Squared, beta):
    X = np.zeros(10)
    step_Size = 0.01
    result = []
    X_LB = [9, 12, 15, 15, 19, 19, 25, 65, 152, 154]
    lower_Bound = calculate_Fx(X_LB, alpha)
    while(True):
        
        new_X = np.add(X, calculate_Gradient_Step(step_Size, beta, alpha))
        Fx = calculate_Fx(new_X, alpha)
        constraint = calculate_Constraint(new_X, beta)
        X = new_X
        if constraint <= R_Squared:
            result = new_X
        else:
            break
        
    print('Assignment Result: ', result)
    return result

X_KKT_Real = [9.012, 12.478, 15.041, 15.4867, 19.4977, 19.888, 25.27, 65.51, 152.08, 154.42]
print('\nLower Bound as Calculated from KKT for real values of X - ', calculate_Fx(X_KKT_Real, alpha))


Lower Bound as Calculated from KKT for real values of X -  88015.3173


In [39]:
res = greedy_Ascent(alpha, R_Squared, beta)
print('\nLower Bound after Greedy Assignment - ', calculate_Fx(res, alpha))
X_LB = [9, 12, 15, 15, 19, 19, 25, 65, 152, 154]
print('\nLower Bound as Calculated from KKT - ', calculate_Fx(X_LB, alpha))

Assignment Result:  [  8.89777778  12.32        14.85        15.29        19.25
  19.635       24.948       64.68       150.15       152.46      ]

Lower Bound after Greedy Assignment -  86898.18988888891

Lower Bound as Calculated from KKT -  87441


#### Here we get the result as [8.89777778, 12.32, 14.85, 15.29, 19.25, 19.635, 24.948, 64.68, 150.15, 152.46] where lower bound is 86898.18988888891. 

### Using the above solutions, implement a branch and bound algorithm to solve the original problem. What is the maximum value? How many complete variable assignments did you have to visit in order to discover the optimum?

In [51]:
def partial_Assignment(index, X, R_Squared, beta):
    rhs = R_Squared
    for i in range(index):
        rhs -= beta[i]*X[i]*X[i]
    
    X[index] = mt.floor(mt.sqrt(rhs/beta[index]))
    
    X = calculate_Xi_Index(alpha, beta, R_Squared, index+1, X)
    
    return X
    
    
def branch_And_Bound(alpha, beta, R_Squared):
    index = 0
    result = []
    initial_X = calculate_Xi_Index(alpha, beta, R_Squared, index, [0]*10)
    global_LowerBound = get_Lower_Bound(initial_X)
    part_Assign = partial_Assignment(index, [0]*10, R_Squared, beta)

    comp_Var_Assignment = 0
    while(True):
        upper_Bound = get_Upper_Bound(part_Assign)
        
        if(upper_Bound < global_LowerBound):
            part_Assign[index] -= 1
            if(part_Assign[0] < 0):
                break
            part_Assign = calculate_Xi_Index(alpha, beta, R_Squared, index+1, part_Assign)

            if(part_Assign[index] == 0):
                if(index != 0):
                    index -= 1
                else:
                    index = 0
            continue
        else:
           
            if(index < len(alpha)-1):
                index += 1

                part_Assign = partial_Assignment(index, part_Assign, R_Squared, beta)

                
            else:
                comp_Var_Assignment += 1
                if upper_Bound == global_LowerBound:
                    temp = part_Assign.copy()
                    result.append(temp)

                else:
                    global_LowerBound = upper_Bound
                    temp = part_Assign.copy()
                    result = [temp]
                if temp[0] == 0:
                    break
            
                part_Assign[index] -= 1
    
    return result, comp_Var_Assignment
    

In [52]:
x_Branch_And_Bound, complet_Var_Assignment = branch_And_Bound(alpha, beta, R_Squared)
print('Assignment using Branch and Bound Algorithm - ', x_Branch_And_Bound)
print('\n Number of Complete Variable Assignments - ', complet_Var_Assignment)


Assignment using Branch and Bound Algorithm -  [[9, 13, 15, 16, 20, 20, 25, 65, 152, 154]]

 Number of Complete Variable Assignments -  424


In [48]:
for i in range(len(x_Branch_And_Bound)):
    max_Val = np.matmul(alpha, x_Branch_And_Bound[i])
    print('Maximum Value - ', max_Val)

Maximum Value -  88011
